# Ethnocentrism in Romeo and Juliet
### Created by Joshua Mateer

Welcome to a case study where we study how ethnocentrism plays a role in Romeo and Juliet. This is still very much a work in progress, but I hope you enjoy it! Let's start by importing the necessary libraries.

In [3]:
# Import necessary libraries
import random
import math

Here are some parameters that the program will use to generate the dialogue. You can edit everything in the cell below and the "story" will adjust accordingly. 

In [4]:
# Your choice does not matter for this. I will change it to whatever I feel like later anyway
goalIndex = 0

# probablities for each class
OUT_HELP_ETHNO = 0.2 # How likely is a ethnocentric agent to help someone outside their group?
IN_HELP_ETHNO = 0.975 # How likely is a ethnocentric agent to help someone within their group?
OUT_HELP_COSMO = 0.95 # How likely is a cosmopolitan agent to help someone outside their group?
IN_HELP_COSMO = 0.3 # How likely is a cosmopolitan agent to help someone within their group?
OUT_HELP_EGO = 0.1 # How likely is a egotistic agent to help someone outside their group?
IN_HELP_EGO = 0.1 # How likely is a egotistic agent to help someone within their group?
OUT_HELP_ALTRU = 0.95 # How likely is a altruistic agent to help someone outside their group?
IN_HELP_ALTRU = 0.925 # How likely is a altruistic agent to help someone within their group?

Don't mind this. I'm just turning all the data you just gave me into a list that the program can read. Reduces code later and allows the program to scale if I want to add/change something later.

In [5]:
# Don't edit this. This is how the program creates the variable conflict system
OPD = [[OUT_HELP_ETHNO, IN_HELP_ETHNO],[OUT_HELP_COSMO, IN_HELP_COSMO],[OUT_HELP_EGO, IN_HELP_EGO],[OUT_HELP_ALTRU, IN_HELP_ALTRU]]

Let's create our model! This model features four custom objects: a Dialogue class that manages individual lines of dialogue, a DialogueManager class that will manage all dialogue and intentions for a character, an Agent class that stores data for our indivdual characters (including their DialogueManager), and a ModelManager class that will dictate how characters interact and keep track of their interactions.

In [143]:
# A new object that will help in dialogue generation
class Dialogue:
    speaker = ""
    text = ""
    addressee = ""
    chapter = 0
    connotations = []

    def __init__(self, speaker, text, addressee, chapter):
        self.speaker = speaker
        self.text = text
        self.addressee = addressee
        self.chapter = chapter
        self.connotations = []
        
    def addConnotation(self, conType):
        self.connotations.append(conType)
        
    def updateTarget(self, target):
        self.addressee = target;

# A new object that will manage each character's dialogue
class DialogueManager:
    library = []

    def __init__(self):
        self.library = []
    
    # add a piece of dialogue to this agent's library
    def add(self, dialogue):
        self.library.append(dialogue)
    
    #clear the dialogueManager
    def clearLibrary(self):
        self.library.clear()
    
    # find a dialgoue with a given character as its addressee
    def findInteraction(self, target, chapter, conType):
        if len(self.library) == 0:
            return "No, my good lord."
        choices = []
        if chapter == -1 and content == "":
            choices = [i for i, d in enumerate(self.library) if d.addressee == target.name]
        elif chapter == -1:
            choices = self.getAllWithConnotation(conType)
        elif conType == "":
            choices = self.getAllInChapter(chapter)
        else:
            choices = self.searchFor(chapter, conType)
        if len(choices) > 0:
            return random.choice(choices).text
        return random.choice(self.library).text
    
    # get the dialogue object at a certain index
    def get(self, n):
        return self.library[n%len(self.library)]
    
    # get all pieces of dialogue in a given chapter
    def getAllInChapter(self, chapter):
        return [i for i, d in enumerate(self.library) if d.chapter == chapter]
    
    # get all pieces of dialogue with a given connotation
    def getAllWithConnotation(self, conType):
        return [i for i, d in enumerate(self.library) if conType in d.connotations]
    
    # get all pieces of dialogue in a given chapter with a given connotation
    def searchFor(self, chapter, conType):
        chapDi = self.getAllInChapter(chapter)
        conDi = self.getAllWithConnotation(conType)
        return [i for i, d in enumerate(self.library) if d in chapDi and d in conDi]
    
    # return the size of this agent's library
    def size(self):
        return len(self.library)
    
# A new object that will dictate how the characters interact
class Agent:
    name = ""
    types = []
    moTypes = []
    dialogue = 0 # null in python, will be turned into a DialogueManager later

    # initalization
    def __init__(self, name, types, moTypes):
        self.name = name
        self.types = types
        self.moTypes = moTypes
        self.dialogue = DialogueManager()

    # add a piece of dialogue to this character's library
    def addDialogue(self, text, target, chapter):
        self.dialogue.add(Dialogue(self.name, text, target, chapter))
    
    # make the dialogue library empty
    def clearDialogue(self):
        self.dialogue.clearLibrary()
        
    # decide whether this agent will help another agent
    def decideToHelp(self, target, content):
        gi = ["family", "gender", "age"].index(content)
        threshold = 0
        if self.types[gi] == target.types[gi]:
            threshold = OPD[self.moTypes[gi]][1]
        else:
            threshold = OPD[self.moTypes[gi]][0]
        return (decision := random.uniform(0,1)) < threshold
    
    # get whichever dialogue was most recently added to this characters' library
    def getMostRecentDialogue(self):
        return self.dialogue.get(-1)
    
    # create an converstaion between this character and a target
    def speakWith(self, target, chapter, content):
        print(self.name+": "+self.dialogue.findInteraction(target, chapter, content))
    
    # updates how willing a character is with respect to a given conflict
    def updateLoyalties(self, number, newVal):
        self.moTypes[number] = newVal;
    
# This is the object that will make the magic happen. Hooray.
class ModelManager:
    agents = []
    
    def __init__(self):
        self.agents = []

    # add an agent to the model
    def add(self, agent):
        self.agents.append(agent)
        
    # find what actor a specific dialogue is addressed to
    def identifyAgent(self, string):
        for agent in self.agents:
            if string.find(agent.name+".\n") != -1:
                return agent;
        return -1;
    
    # return the size of the agents array
    def size(self):
        return len(self.agents)
    
    #remove an agent from the sim
    def remove(self, agentName):
        i = 0
        while self.agents[i].name != agentName and i < len(self.agents):
            i += 1
        self.agents.pop(i)
                
    # create an interaction between two agents
    def createInteraction(self, agent1, agent2, chapter, content):
        agent1.speakWith(agent2, chapter, content)
        agent2.speakWith(agent1, chapter, content)
        agent1agrees = agent1.decideToHelp(agent2, content)
        agent2agrees = agent2.decideToHelp(agent1, content)
        if agent1agrees and agent2agrees:
            print("\nHooray! "+agent1.name+" and "+agent2.name+" had a positive interaction!")
        elif agent1agrees or agent2agrees:
            print("\nHuh. "+agent1.name+" and "+agent2.name+" had a neutral interaction.")
        else:
            print("\nOh no! "+agent1.name+" and "+agent2.name+" had a negative interaction.")
        print("\n---------------------------------------------------\n")
    
    # run the simulation one time
    def runSim(self, chapters, goal):
        # assign our goalIndex
        goalIndex = 0 if goal == "family" else 1 if goal == "gender" else 2
        for c in chapters:
            random.shuffle(self.agents)
            print("Simulating Act "+str(c)+"...")
            print("Generating "+goal+" interactions...\n")
            for i in range(0, math.floor(len(self.agents)/2)):
                # create an interaction between each pair of characters in the story.
                self.createInteraction(self.agents[i*2], self.agents[i*2+1], c, goal)

At this point, we're finally ready to make some characters! Here are some sample characters I made. Each character has a set of attributes that the program can access to check interactions. Additionally, each character has a set of numbers that represents how they will interact with other characters in a given conflict situation.  
Key:
- a 0 represents that they are ethnocentric with respect to this conflict
- a 1 represents that they are cosmopolitan with respect to this conflict
- a 2 represents that they are egotistic with respect to this conflict
- a 3 represents that they are altruistic with respect to this conflict

- The first number represents their response to family conflict
- The second number represents their response to gender conflict
- The third number represents their response to age conflict

In [150]:
mod = ModelManager()

# Let's create all the different characters and set how they'll interact
# Feel free to edit the numbers at the end to change how they interact with different groups of people
mod.add(Agent("MERCUTIO", ["Montague", "male", "young"], [0, 3, 2]))
mod.add(Agent("MONTAGUE", ["Montague", "male", "old"], [2, 2, 2]))
mod.add(Agent("ROMEO", ["Montague", "male", "young"], [3, 3, 0]))
mod.add(Agent("BENVOLIO", ["Montague", "male", "young"], [0, 3, 2]))
mod.add(Agent("ABRAM", ["Montague", "male", "young"], [2, 3, 2]))
mod.add(Agent("BALTHASAR", ["Montague", "male", "young"], [0, 3, 2]))
mod.add(Agent("CAPULET", ["Capulet", "male", "old"], [2, 2, 2]))
mod.add(Agent("JULIET", ["Capulet", "female", "young"], [3, 3, 0]))
mod.add(Agent("TYBALT", ["Capulet", "male", "young"], [0, 2, 2]))
mod.add(Agent("NURSE", ["Capulet", "female", "old"], [3, 3, 1]))
mod.add(Agent("PETER", ["Capulet", "male", "young"], [3, 3, 3]))
mod.add(Agent("SAMPSON", ["Capulet", "male", "young"], [2, 2, 2]))
mod.add(Agent("GREGORY", ["Capulet", "male", "young"], [2, 2, 2]))
mod.add(Agent("FRIAR LAWRENCE", ["Neither", "male", "old"], [3, 3, 3]))
mod.add(Agent("PRINCE", ["Neither", "male", "old"], [0, 3, 3]))

We want to be able to analyze a whole bunch of different conflicts, so let's set up a list of words that suggest a relevance to different types of conflict.

In [151]:
# what words signify a familial conflict?
familyWords = ["Montague", "Capulet", "family"]

# what words signify a gender conflict?
genderWords = ["men", "women", "man", "woman", "boy", "girl", "maid", "lady", "ladies", "love"]

# what words signify an age conflict?
ageWords = ["age", "father", "mother", "son", "daughter", "old", "young"]


Finally, we're ready to actually read Romeo and Juliet! We will read in the file, walk through our character list and find all of their dialogoue. Eventually, I'd like to figure out how to have the model predict who each line is addressed to, but right now that doesn't work.

In [152]:
# Let's get some dialogue set up.

#The dialogue library carrys over between runs, so let's just clear that out now
for agent in mod.agents:
    agent.clearDialogue()

# Open the text file 
play = open('romeo_juliet.txt')

# Read the text from the file, then split the text into acts
play_text = play.read().split("\n\n\n\n")[2:6]

for actNum, act in enumerate(play_text):
    # split each act into individual dialogue lines
    lines = act.split("\n\n")
    for i in range(len(lines)):
        # for each line, identify the speaker and who they're speaking to
        # then add their dialogue into the library for that character
        actor = mod.identifyAgent(lines[i])
        if i < len(lines)-1:
            target = mod.identifyAgent(lines[i+1])
        else:
            target = mod.identifyAgent(lines[i-1])
        if actor != -1:
            if target != -1:
                actor.addDialogue(lines[i].replace(actor.name + '.\n', ''), target.name, actNum+1)
            else:
                actor.addDialogue(lines[i].replace(actor.name + '.\n', ''), "unknown", actNum+1)
            dia = actor.getMostRecentDialogue()
            # do we think this dialogue is related to a family conflict?
            if any(word in dia.text for word in familyWords):
                dia.addConnotation("family")
            # do we think this dialogue is related to a gender conflict?
            if any(word in dia.text for word in genderWords):
                dia.addConnotation("gender")
            # do we think this dialogue is related to a age conflict?
            if any(word in dia.text for word in ageWords):
                dia.addConnotation("age")

## The Simulation

Now we're finally ready to run the sim! Change the `chapters` variable to add different acts into the story, or change the `conflict` variable to change which conflict is being addressed. Then just press run on the cell below to watch the story unfold before your very eyes. 

In [154]:
# Just press run on this cell to see an entirely new story unfold!
# You can also change these variables to test different conflicts
acts = [1]
conflict = "family"

mod.runSim(acts, conflict)

Simulating Act 1...
Generating family interactions...

MERCUTIO: Thou art like one of these fellows that, when he enters the confines of
a tavern, claps me his sword upon the table, and says 'God send me no
need of thee!' and by the operation of the second cup draws him on the
drawer, when indeed there is no need.
PETER: Prates. What say you, Hugh Rebeck?

Huh. MERCUTIO and PETER had a neutral interaction.

---------------------------------------------------

ROMEO: By a name
I know not how to tell thee who I am:
My name, dear saint, is hateful to myself,
Because it is an enemy to thee.
Had I it written, I would tear the word.
PRINCE: Benvolio, who began this bloody fray?

Huh. ROMEO and PRINCE had a neutral interaction.

---------------------------------------------------

BALTHASAR: I have no dialogue because Josh can't write a parsing method properly.
BENVOLIO: This wind you talk of blows us from ourselves:
Supper is done, and we shall come too late.

Hooray! BALTHASAR and BENVOLIO 